In [1]:
!pip install newspaper3k
!pip install tqdm

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import json
import requests
import my_settings
import pandas as pd
from datetime import datetime
import urllib
from tqdm import tqdm
from newspaper import Article 

In [3]:
datelist = pd.date_range(start="2022-01-02", end="2022-12-31").tolist()
datelist = [int(date.strftime("%Y%m%d")) for date in datelist]

In [4]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

In [5]:
def get_soup(date, page, headers):
    """
  Input
  - date(int): date in integer type
  - page(int): number of page
  - headers(dict): dictionary of headers for scraping

  Output
  - soup(bs4.Soup)
  """
    url = f"https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&listType=paper&date={date}&page={page}"

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    return soup

In [6]:
def get_links_list(x):
    """
    function to get list of links in the current url

    Input
    - x: soup
    Output
    - links: list of links in current url
    """

    headline = x.find_all('dl')

    links = []
    for item in headline:
        try:
            link = item.find('dt', {'class': 'photo'}).find('a')['href']
            links.append(link)
        except:
            link = item.find('dt', {'class': None}).find('a')['href']
            links.append(link)

    return links


In [7]:
news_dict = {}
date_count = 0
for date in tqdm(datelist):
    url_date = f"https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&listType=paper&date={date}"
    response = requests.get(url_date, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    link_count = len(soup.select('#main_content > div.topbox_type6 > div > ul > li > a'))
    if link_count == 0:
        continue
    date_count += 1
    page = 1
    links = []
    while page <= link_count:
        try:
            soup = get_soup(date, page, headers)
            new_links = get_links_list(soup)
            if not new_links:
                break
            for link in new_links:
                response = requests.get(link, headers=headers)
                soup = BeautifulSoup(response.text, 'html.parser')
                if soup.select_one('#_LNB > ul > li.Nlist_item._LNB_ITEM.is_active > a > span') and \
            '경제' in soup.select_one('#_LNB > ul > li.Nlist_item._LNB_ITEM.is_active > a > span').text:
                    links.append(link)
            page += 1
        except:
            break
    news_dict[date] = links

100%|███████████████████████████████████████| 364/364 [3:33:32<00:00, 35.20s/it]


In [8]:
total_links = 0
len_list = []
for links in news_dict.values():
    len_list.append(len(links))
    total_links += len(links)
avg_links = total_links / date_count

print(f"총 링크 개수: {total_links}, 뉴스 기사 있는 날짜 수 : {date_count}, 평균 일 별 기사 수 : {avg_links}")

총 링크 개수: 20444, 뉴스 기사 있는 날짜 수 : 312, 평균 일 별 기사 수 : 65.52564102564102


In [9]:
# csv파일로 만들기 위해 데이터 프레임 화
max_length = max([len(v) for v in news_dict.values()])
for k, v in news_dict.items():
    if len(v) < max_length:
        news_dict[k] += [None] * (max_length - len(v))
news_df = pd.DataFrame(news_dict)

In [10]:
news_df.to_csv('news2022.csv')

In [11]:
news2022 = pd.read_csv('news2022.csv')

In [13]:
news_df

,20220103,20220104,20220105,20220106,20220107,20220108,20220110,20220111,20220112,20220113,...,20221221,20221222,20221223,20221224,20221226,20221227,20221228,20221229,20221230,20221231
0,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...
1,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...
2,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...
3,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/009/000...,https://n.news.naver.com/mnews/article/00

In [14]:
df = news2022.loc[:, '20220103':'20221231']
# 월별로 데이터프레임을 만들어 저장
for month in range(1, 13):
    month_str = str(month).zfill(2)
    df_month = df.loc[:, df.columns.str.contains(f'2022{month_str}')].copy(deep=True)
    df_month.to_csv(f'news2022{month_str}.csv', index=False)